<a href="https://colab.research.google.com/github/ambarishg/open-source-llms/blob/main/INSTRUCT_MISTRAL_7B_AND_QDRANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install qdrant-client

In [50]:
!pip install sentence_transformers

In [51]:
!pip install ctransformers[cuda]

In [52]:
URL = "http://20.120.85.148:6333"
MODEL_NAME = "all-MiniLM-L6-v2"
DOCUMENTS_COLLECTION_NAME="RESEARCH_PAPERS"

In [53]:
import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *

In [54]:
client = qc.QdrantClient(url=URL)
METRIC = qmodels.Distance.COSINE

In [55]:
from sentence_transformers import SentenceTransformer

def load_model():
    return SentenceTransformer(MODEL_NAME)

In [56]:
def get_search_document_results(user_input, CATEGORY="") -> str:
    model = load_model()
    xq = model.encode(user_input,convert_to_tensor=True)
    if CATEGORY == '':
        query_filter = None
    else:
        query_filter=qmodels.Filter(
            must= [
                FieldCondition(
                    key="Category",
                    match=models.MatchValue(value=CATEGORY),
                )
            ],
        )

    search_result = client.search(collection_name=DOCUMENTS_COLLECTION_NAME,
                                    query_vector=xq.tolist(),
                                    query_filter=query_filter,
                                    limit=3)
    contexts =""
    for result in search_result:
        contexts +=  result.payload['token']+"\n---\n"
    return(contexts)

In [57]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                                           model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                                           model_type="mistral", gpu_layers=50)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
query = "What is attention?"

context = get_search_document_results(query)

In [59]:
print(context)

 This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position ican depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors
---
 Can active memory replace attention? In Advances in Neural
Information Processing Systems, (NIPS) , 2016.
10[21] Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attention-
based neural machine translation. arXiv preprint arXiv:1508.04025 , 2015.
[22] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention
model. In Empirical Methods in Natural Language Processing , 2016.
[23] Romain Paulus, Caiming Xiong, and Richard Socher
---
 Not only do individual attention
heads clearly learn to perform different tasks, many appear to exhibit behavior related to the

In [60]:
prompt = f"""Answer based on context:\n\n{context}\n\n{query}"""
prompt2 = f"<s>[INST] {prompt} [/INST]"
print(prompt2)

<s>[INST] Answer based on context:

 This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position ican depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors
---
 Can active memory replace attention? In Advances in Neural
Information Processing Systems, (NIPS) , 2016.
10[21] Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attention-
based neural machine translation. arXiv preprint arXiv:1508.04025 , 2015.
[22] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention
model. In Empirical Methods in Natural Language Processing , 2016.
[23] Romain Paulus, Caiming Xiong, and Richard Socher
---
 Not only do individual attention
heads clearly learn to perform different tasks, many appea

In [61]:
%%time
print(llm(prompt = prompt2,temperature=0.1,max_new_tokens=1024))

 Attention is a function that maps a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. It can be used in natural language processing tasks such as machine translation to help the model focus on relevant information when making predictions.
CPU times: user 31.7 s, sys: 4.4 s, total: 36.1 s
Wall time: 27.1 s
